# HW1 - Intro to the Map Reduce Paradigm  
__ `MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Spring 2018`__

Welcome to Machine Learning at Scale! This first homework assignment introduces one of the core strategies in distributed processing: divide and conquer. We'll use the simplest of tasks, word counting, to illustrate the difference between a scalable and non-scalable algorithm. You will be working with the text of _Alice in Wonderland_ to put these ideas into practice using Python and Bash scripting. By the end of this week you should be able to:
* ... __describe__ the Bias-Variance tradeoff as it applies to Machine Learning.
* ... __explain__ why we consider word counting to be an "Embarrassingly Parallel" task.
* ... __estimate__ the runtime of embarrassingly parallel tasks using "back of the envelope" calculations.
* ... __implement__ a Map Reduce algorithm using the Command Line.
* ... __set-up__ a Docker container and know why we use them for this course.

You will also  become familiar (if you aren't already) with `defaultdict`, `re` and `time` in Python, linux piping and sorting, and Jupyter magic commands `%%writefile` and `%%timeit`. __Please refer to the `README` for detailed submission instructions__.

__IMPORTANT:__ If you're not familiar with linux, you should read the following tutorial reagrding **piping** and **redirecting**: https://ryanstutorials.net/linuxtutorial/piping.php You will need to understand the differences to answer some of the later questions.


# Notebook Set-Up
Before starting your homework run the following cells to confirm your setup.

In [1]:
# confirm you are running Python 3
import sys
sys.version_info

sys.version_info(major=3, minor=7, micro=5, releaselevel='final', serial=0)

In [2]:
# imports
import re
import sys

Create a folder for any data you download locally.

In [4]:
!mkdir data
# NOTE: the contents of this directory will be ignored by git.

mkdir: cannot create directory `data': File exists


# Question 1: Introductions

`The Caterpillar and Alice looked at each other for some time in silence: at last the Caterpillar took the hookah out of its mouth, and addressed her in a languid, sleepy voice. "Who are you?" said the Caterpillar.`   
<div style="text-align: right"> -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 4 </div>


__Tell us about yourself! Briefly describe where you live, how far along you are in MIDS, what other classes you are taking and what you want to get out of w261.__

> I am currently live in Austin, TX. This is my 4th semester in MIDS and I am taking w209 Data visualization with w261. I am Environmental Engineer and I have been very interested in the field of Data Science while I was doing air modeling in school. I did pretty good at w205 where we have got exposure to Spark and Hadoop, but I would like to learn more about Hadoop and Spark so that I will be able to apply it in my future work.

# Question 2: Bias - Variance
__In 1-2 sentences explain the bias-variance trade off. Describe what it means to "decompose" sources of error. How is this used in machine learning?__ Please use mathematical equation(s) to support your explanation. (Use `$` signs to take advantage of $\LaTeX$ formatting, eg. `$f(x)$` will look like: $f(x)$). Please also cite any sources that informed your answer.

> Variances measures how inconsistent predictions from each other, whereas bias is introduced with model selection. 
A tradeoff between bias and variance that comes with model complexity: models that are too complex will have high variance and low bias; models that are too simple will have high bias and low variance. The best model will have both low bias and low variance.

> The sources of error can be decomposed into : $$Error = Estimator Variance + Squared Estimator Bias + Noise$$. 
 The mathematic formula is:  $$ E(g(x^*) - y^*)^2] = E[(g(x^*) - E[g(x^*)]^2] + (E([g(x^*)]-f(f(x^*))^2 + E[(y^*-f(x^*))^2]$$

> This is insight is very helpful in machine learning application when choosing the best model. The complexity of a prediction model will have an optimum point in terms of minimize the error. 

# Question 3: Tokenizing
A number of our assignments this term will involve extracting information from text. A common preprocessing step when working with raw files is to 'tokenize' (i.e. extract words from) the text. Within the field of Natural Language Processing a lot of thought goes into what specific tokenizing makes most sense for a given task. For example, you might choose to remove punctuation or to consider punctuation symbols  'tokens' in their own right. __In this question you'll use the Python `re` module to create a tokenizer to use when you perform WordCount on the _Alice In Wonderland_ text.__

### Q3 Tasks:

* __a) short response:__ In the Naive Bayes algorithm (which we'll implement next week), we'll estimate the _likelihood_ of a word by counting the number of times it appears and dividing by the size of the vocabulary (total number of unique words). Using the text: *"Alice had an adventure that took alice to wonderland"*, give a concrete example of how two different tokenizers could cause us to get two different results on this calculation. [`HINT`: _you should not need to read up on Naive Bayes to answer this question_]  
  

* __b) short response:__ When tokenizing in this assignment we'll remove punctuation and discard numerical digits by making everything lowercase and then capturing only consecutive letters a to z. Suppose __`tokenize(x)`__ is a Python function that performs the desired tokenization. What would __`tokenize("By-the-bye, what became of Alice's 12 hats?!")`__ output? Type the answer in the space provided below.


* __c) code:__  Fill in the regular expression pattern in the cell labeled `part c` so that the subsequent call to `re.findall(RE_PATTERN, ...)` returns the tokenization described above. [`HINT`: _we've taken care of the lowercase part for you. If regex is new to you, check out the [`re`  documentation](https://docs.python.org/3/library/re.html) or [this PyMOTW tutorial](https://pymotw.com/2/re/)._]

### Q3 Student Answers:
> __a)__ The tokenizer will convert words from the text into tokens. Different tokenizers will extract the words differently from the text. Therefore, when different words from the same text was tokenized, the ratio of words from text over the vocabulary will be different. For example, if we did not use lower case, the word "alice" and "Alice" will be counted as two different tokens, and the ratio of number of "alice" over total vocabulary will be different.

> __b)__ It will be: ['by', 'the', 'bye', 'what', 'became', 'of', 'alice', 's', 'hats']

In [20]:
# Part C - Fill in the regular expression
RE_PATTERN = re.compile(r'[a-z]+')          

In [21]:
# Tokenize - DO NOT MODIFY THIS CELL, just run it as is to check your pattern
words = re.findall(RE_PATTERN, "By-the-bye, what became of Alice's 12 hats?!".lower())
print(words)

['by', 'the', 'bye', 'what', 'became', 'of', 'alice', 's', 'hats']


# Load the Data
` "Please would you tell me", said Alice, a little timidly, for she was not quite sure whether it was good manners for her to speak first, "why your cat grins like that?"`  
<div style="text-align: right">  -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 4  </div>

For the main part of this assignment we'll be working with the free plain text version of _Alice's Adventures in Wonderland_ available from Project Gutenberg. __Use the first two cells below to download this text and preview the first few lines.__ 

In [7]:
# Download Full text 
# NOTE: feel free to replace 'curl' with 'wget' or equivalent command of your choice.
!curl "http://www.gutenberg.org/files/11/11-0.txt" -o data/alice.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  170k  100  170k    0     0   260k      0 --:--:-- --:--:-- --:--:--  260k


In [3]:
# Take a peak at the first few lines
!head -n 6 data/alice.txt
# NOTE: If you are working in JupyterLab on Docker you may not see the output 
# below due to an encoding issue... in that case, use a terminal on Docker to 
# execute this head command and confirm that the file has downloaded properly, 
# this encoding issue should not affect your work on subsequent HW items.

The Project Gutenberg EBook of Alice’s Adventures in Wonderland, by Lewis Carroll

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org


We'd also like you to develop a habit of creating small files with simulated data for use in developing, debugging and testing your code. The jupyter magic command `%%writefile` is a convenient way to do this. __Run the following cells to create a test data file for use in our word counting task.__

In [29]:
%%writefile data/alice_test.txt
This is a small test file. This file is for a test.
This small test file has two small lines.

Overwriting data/alice_test.txt


In [4]:
# confirm the file was created in the data directory using a grep command:
!ls data | grep test

alice_test.txt
alice_test.txt.output


# Question 4: Word Count in Python

Over the course of this term you will also become very familiar writing Python programs that read from standard input and using Linux piping commands to run these programs and save their output to file. __In this question you will write a short python script to perform the Word Count task and then run your script on the _Alice in Wonderland_ text__. You can think of this like a 'baseline implementation' that we'll later compare to the parallelized version of the same task.

### Q4 Tasks:

* __a) code:__ Complete the Python script in the file __`wordCount.py`__. Read the docstrings carefully to be sure you understand the expected behavior of this function. Please do not code outside of the marked location.


* __b) testing:__ Run the cell marked `part b` to call your script on the test file we created above. Confirm that your script returns the correct counts for each word by visually comparing the output to the test file. 


* __c) results:__ When you are confident in your implementation, run the cell marked `part c` to count the number of occurrences of each word in _Alice's Adventures in Wonderland_. In the same cell we'll pipe the output to file. Then use the provided `grep` commands to check your answers.


* __d) short response:__ Suppose you decide that you'd really like  a word and its plural (e.g. 'hatter' and 'hatters' or 'person' and 'people') to be counted as the same word. After we have run the wordcount would it be more efficient to post-process your output file or discard your output file and start the analysis over with a new tokenizer? Explain your reasoning briefly. 

### Q4 Student Answers:
> __a-c)__ _Complete the coding portions of this question before answering 'e'._

> __d)__ It will be much easier and efficient to post-process the output by merge sort the words and check if the next word is the plural form and then add up or using dictionary to merge keys. Since if we use different mappers, we will have to merge the splitted files anyway, it would make more sense to post-process.

In [ ]:
# part a - DO YOUR WORK IN wordCount.py

In [31]:
# part b - DO NOT MODIFY THIS CELL, just run it as is to test your script
!python wordCount.py < data/alice_test.txt

this	3
is	2
a	2
small	3
test	3
file	3
for	1
has	1
two	1
lines	1


In [32]:
# part c - DO NOT MODIFY THIS CELL, just run it as is to perform the word count.
!python wordCount.py < data/alice.txt > data/alice_counts.txt

Take a look at the first 10 words & their counts.

In [33]:
!head data/alice_counts.txt

the	1832
project	88
gutenberg	98
ebook	11
of	635
alice	404
s	224
adventures	12
in	435
wonderland	8


__Check your results:__ How many times does the word "alice" appear in the book? 

In [34]:
# EXPECTED OUTPUT: 403
!grep alice data/alice_counts.txt

alice	404


__Check your results:__ How many times does the word "hatter" appear in the book? 

In [35]:
# EXPECTED OUTPUT: 56
!grep hatter data/alice_counts.txt

hatter	56
hatters	1


__Check your results:__ How many times does the word "queen" appear in the book? 

In [36]:
# EXPECTED OUTPUT: 75
!grep queen data/alice_counts.txt

queen	76
queens	1


# Question 5: Unix Sorting
Another common task in this course's assignments will be to make strategic use of sorting. To illustrate what we mean by 'strategic,' let's return to the (admitedly artificial) scenario posed in Question 4d): _we have a file of word counts but we want to find all pairs of words and plurals (eg. 'queen' and 'queens') to join their counts into a single record_. Suppose we were going to do this by hand. A logical approach would be to start with the first word and read through each subsequent word to check if they are a singular-plural pair, then repeat this process for each word in the file.    

__IMPORTANT NOTE__: For this question, pretend that all plurals simply end with 's'.

### Q5 Tasks:
* __a) short response + code:__ In the worst case scenario, how many comparisons will we have to make to perform the process described above? Use $\LaTeX$ formatting to show the calculations that led to your answer. What is the Big O complexity of this "algorithm"? [`HINTs:` _To answer the first part of this question you'll want to know how many words are in your_ `alice_counts.txt` _file -- in the space provided below, write a unix command to check. If you need a Big O notation refresher, here's_ a [blog post](https://rob-bell.net/2009/06/a-beginners-guide-to-big-o-notation/), a [cheatsheet](http://bigocheatsheet.com), and a [thorough explanation](http://pages.cs.wisc.edu/~vernon/cs367/notes/3.COMPLEXITY.html).]


* __b) short response:__ If the word count file were sorted alphabetically, how would your approach to this task change? How many comparisons would you have to make? What would the Big O complexity of this new strategy be? *(Remember that for this question, we're assuming all plural words simply end with 's')*


* __c) short response:__ What is the Big O complexity of the fastest sorting algorithms? When we take the time to sort into account, does it make sense to alphabetize before combining singular-plural pairs? [`HINT:` *When it comes to sorting algorithms -- Wikipedia is your friend, as is the cheatsheet from part a.*]


* __d) code:__ Write a unix command to sort your word count file alphabetically. Save (i.e. [redirect](https://superuser.com/questions/277324/pipes-vs-redirects)) the results to `data/alice_counts_A-Z.txt`. [`HINT:` if Unix sort commands are new to you, start with [this biowize blogpost](https://biowize.wordpress.com/2015/03/13/unix-sort-sorting-with-both-numeric-and-non-numeric-keys/) or [this unixschool tutorial](http://www.theunixschool.com/2012/08/linux-sort-command-examples.html)]


* __e) code:__ A (slightly) more likely scenario is that we'd want to look at the top 10 most frequent words that appear in the book. Write a unix command to sort your word count file from highest to lowest count. Save (i.e. [redirect](https://superuser.com/questions/277324/pipes-vs-redirects)) your results to `data/alice_counts_sorted.txt`; then run the provided cell to print the top ten words. Compare your output to the expected output we provide.

### Q5 Student Answers:
> __a)__ In the worst case scenario, assumming there are n words in the document and for each word, it will find its pair of plural after scan through the entire document and then merge or not merge if no plural exists. Therefore, for the first word, it will compare with all the words, n-1 times and find no plural form. The second word, will scan n-2 times ... so on. The total comparisons count will be $ n-1 + n-2 + ...+1 = (n)*(n-1)/2$. The complexity of the algorithm will be $O(n^2)$. As we knew from part a below, n = 3019 , so the total comparision will be 4555671 times

> __b)__ If the document is sorted already, then we can only compare with the next word instead of scanning through the entire file. In the worst case scenario, no plurals will be merged, every word will be compared with the next word, there will only be $n-1$ times of comparison. The complexity of the algorithm will be $O(n)$. As we knew from part a below, n = 3019 , so the total comparision will be 3018 times.

> __c)__ The fastest sorting algorithm is merge sort and the Big O complexity is $O(nlogn)$. It totally make sense to aphabetize before combining singular-plural pairs, since sorting-$O(nlogn)$ is higher than combining-$O(n)$, the total time will still be $O(nlogn)$ which is way faster than $O(n^2)$ as the n gets bigger. 


In [9]:
# part a - write a unix command to check how many records are in your word count file
! wc -l data/alice_counts.txt # each line has one record with one word and one number

3019 data/alice_counts.txt


In [10]:
# part d - unix command to sort your word counts alphabetically 
! sort -k1,1 data/alice_counts.txt > data/alice_counts_A-Z.txt

In [11]:
# part d - DO NOT MODIFY THIS CELL, run it as is to confirm your sort worked
!head data/alice_counts_A-Z.txt   

a	695
abide	2
able	1
about	102
above	3
absence	1
absurd	2
accept	1
acceptance	1
accepted	2


In [5]:
# part e - unix command to sort your word counts from highest to lowest count
! sort -k2,2nr data/alice_counts.txt > data/alice_counts_sorted.txt

In [6]:
# part e - DO NOT MODIFY THIS CELL, run it as is to confirm your sort worked
!head data/alice_counts_sorted.txt  

the	1832
and	945
to	810
a	695
of	635
it	610
she	553
i	546
you	484
said	462


<table>
<th>expected output for (d):</th>
<th>expected output for (e):</th>
<tr><td><pre>
a          690
abide      2
able       1
about      102
above      3
absence    1
absurd     2
accept     1
acceptance 1
accepted   2
</pre></td>
<td><pre>
the	 1818
and	 940
to	 809
a	 690
of	 631
it	 610
she	 553
i	 545
you	 481
said	 462
</pre></td></tr>
</table>

# Question 6: Parallel Word Count (part 1)
What would happen if we tried to run our script on a much larger dataset? For one thing, it would take longer to run. However there is a second concern. The Python object that aggregates our counts (`defaultdict`) exists in memory on the machine running this notebook. If the vocabulary is too large for the memory space available we would crash the notebook. The solution? Divide and Conquer! Instead of running the script on the whole dataset at once, we could split our text up in to smaller 'chunks' and process them independently of each other. __In this question you'll use a bash script to "parallelize" your Word Count.__


### Q6 Tasks:
* __a) read provided code:__ The bash script `pWordCount_v1.sh` takes an input file, splits it into a specified number of 'chunks', and then applies an executable of your choice to each chunk. Read through this code and make sure you understand each line before you proceed. [`HINT:` _For now, ignore the 'student code' section -- you'll use that in part c._]


* __b) short response:__ Below we've provided the command to use this script to apply your analysis (`wordCount.py`) to the _Alice_ text in 4 parallel processes. We'll redirect the results into a file called `alice_pCounts.txt.` Run this analysis and compare the count for the word 'alice' to your answer from Question 4. Explain what went wrong and describe what we have to add to `pWordCount_v1.sh` to fix the problem.


* __c) code:__ We've provided a python script, `aggregateCounts_v1.py`, which reads word counts from standard input and combines any duplicates it encounters. Read through this script to be sure you understand how it is written. Then follow the instructions in `pWordCount_v1.sh` to make a one-line modification so that it accepts `aggregateCounts_v1.py` as a 4th argument and uses this script to combine the chunk-ed word counts. Run the cell below to confirm that you now get the correct results for your 'alice' count.

### Q6 Student Answers:
> __b)__ The mapper divided the data into 4 chunks and each chunk counted individually but it did not has a reducer. Therefore, the final output does not combine all the 4 chunks into 1 final file. We will need to add a reducer to `aggregateCounts_v1.py` in order to aggregate the same keys.

In [7]:
# part b - make sure your scripts are executable (RUN THIS CELL AS IS)
!chmod a+x pWordCount_v1.sh
!chmod a+x wordCount.py

In [8]:
# part b - parallel word count on Alice text (RUN THIS CELL AS IS)
!./pWordCount_v1.sh 4 'data/alice.txt' 'wordCount.py' > 'data/alice_pCounts.txt'

In [9]:
# part b - check alice count (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

alice	113
alice	126
alice	122
alice	43


In [10]:
# part c - make sure the aggregateCounts script is executable (RUN THIS CELL AS IS)
!chmod a+x aggregateCounts_v1.py

In [11]:
# part c - parallel word count on Alice text (RUN THIS CELL AS IS)
!./pWordCount_v1.sh 4 'data/alice.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v1.py' \
                   > 'data/alice_pCounts.txt'

In [12]:
# part c - check alice count (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

alice	404


In [61]:
!sort --help

Usage: sort [OPTION]... [FILE]...
  or:  sort [OPTION]... --files0-from=F
Write sorted concatenation of all FILE(s) to standard output.

Mandatory arguments to long options are mandatory for short options too.
Ordering options:

  -b, --ignore-leading-blanks  ignore leading blanks
  -d, --dictionary-order      consider only blanks and alphanumeric characters
  -f, --ignore-case           fold lower case to upper case characters
  -g, --general-numeric-sort  compare according to general numerical value
  -i, --ignore-nonprinting    consider only printable characters
  -M, --month-sort            compare (unknown) < `JAN' < ... < `DEC'
  -h, --human-numeric-sort    compare human readable numbers (e.g., 2K 1G)
  -n, --numeric-sort          compare according to string numerical value
  -R, --random-sort           sort by random hash of keys
      --random-source=FILE    get random bytes from FILE
  -r, --reverse               reverse the result of comparisons
      --sort=WORD             

# Question 7: Parallel Word Count (part 2)
Congratulations, you've just implemented a Map-Reduce algorithm! From here on out, we'll refer to the two python scripts you passed to `pWordCount_v1.sh` as '_mapper_' and '_reducer_'. The bash script itself served as our '_framework_' -- it split up the original input, then ___mapped___ our word counting script on to each chunk, then ___aggregated (a.k.a. reduced)___ the resulting files by piping them into our collation script.  Unfortunately, as you may have realized already, there is a major scalability concern with this particular implementation. __In this question you'll fix our implementation of parallel word count so that it will be scalable.__

__HINT:__ MapReduce uses the Merge-Sort algorithm under the hood. Linux `sort` command has a merge option which you can use to simiulate the MapReduce framework. Use the `man sort` command to find more information on this option. 

### Q7 Tasks:

* __a) short response:__ What is the potential scalability problem with the provided implementation of `aggregateCounts_v1.py`? Explain why this supposedly 'parallelized' Word Count wouldn't work on a really large input corpus.  [`HINT:` _see the intro to Q6_]


* __b) code:__ Fortunately, a 'strategic sort' can solve this problem. Read the instructions at the top of `pWordCount_v2.sh` carefully then make your changes that alphabetically sort the output from the mappers (`countfiles`) before piping them into the reducer script.


* __c) code:__ Write the main part of `aggregateCounts_v2.py` so that it takes advantage of the sorted input to add duplicate counts without storing the whole vocabulary in memory. Refer to the file docstring for more detailed instructions. Confirm that your implementation works by running it on both the test and true data files.


* __d) short response:__ If you are paying close attention, this rewritten reducer sets us up for a truly scalable solution, but doesn't get us all the way there. In particular, while we chunked our data so it can be processed by multiple mappers, we're still streaming the entire dataset through one reduce script. If the vocabulary is too large to fit on a single computer, we might split the word counts in half after sorting them, then perform the reducing on two separate machines. Explain what could go wrong with this approach. (For now, ignore the question of how we'd sort a dataset that is too large to fit on a single machine and just focus on what might be wrong about the result of this split-in-half reducing).


* __e) short response:__ Can you come up with a different way of splitting up the data that would allow us to perform the reducing on separate machines without needing any postprocessing? This is a theoretical question -- don't worry if you don't know how to implement your idea in a bash script, just describe how you'd want to split the sorted counts into different files to be reduced separately.

### Q7 Student Answers:

> __a)__ With all the parallelized word count chunk unsorted and piped them all into the reducer, it can deal with running time concerns. However, the parallelized count dictionaries exists in memory on the machine running this notebook with even more duplicated keys and values when the 4 large dictionaries were merged into 1 file. 

> __b-c)__ _complete the coding portions of this question before answering d and e._

> __d)__ First, if we store all the data chunks into one word count file and then sort them on one machine before splitting into two reducer machines, we might still face the problem of having one word count file being too large for the RAM. 

>	Second, even if we have two sorted word count files in two reducers, how the keys being separate into two machines could potentially be a problem. Since the word count file is sorted, I assume the file will be cut into half and half, but if the cutting point accidentally cut the plural pairs or the same keys, i.e, "queen" and "queens" into two different files or 4 "alice" keys cut into two files, then the output will have two un-reduced mistakes.


> __e)__ I would extract the *unique* keys from the word count dictionary , then separate the keys into two different reducers instead of simply cutting them into half. Or at the splitting point, check the next key word, see if it is different to the spliting point key. For example, if we split n lines of word counts file into half, the middle point of the n/2-th word is "queen", I will check the next word and see if it is the same "queen" or plural form  "queens"; if it is true, then move the splitting point one more until the next word is different.

In [58]:
# part c - make sure your scripts are executable (RUN THIS CELL AS IS)
!chmod a+x pWordCount_v2.sh
!chmod a+x aggregateCounts_v2.py

In [59]:
# part b - parallel word count on Alice text (RUN THIS CELL AS IS)
!./pWordCount_v2.sh 4 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/alice_pCounts.txt'

In [60]:
# part c - test your code on the test file (RUN THIS CELL AS IS)
!./pWordCount_v2.sh 4 'data/alice_test.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v2.py'

a	2
file	3
for	1
has	1
is	2
lines	1
small	3
test	3
this	3
two	1


In [45]:
# part c - run your code on the Alice file (RUN THIS CELL AS IS)
!./pWordCount_v2.sh 4 'data/alice.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v2.py' \
                   > 'data/alice_pCounts.txt'

In [46]:
# part c - confirm that your 'alice' count is correct (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

alice	404


# Question 8: Scalability Considerations
In your reading for Week 1's live session, [Chapter1, section 2](https://lintool.github.io/MapReduceAlgorithms/MapReduce-book-final.pdf) of _Data Intensive Text Processing with MapReduce_, Lin and Dyer discuss a number of "Big Ideas" that underlie large scale processing: __scale "out," not "up"; assume failures are common; move processing to the data; process data sequentially and avoid random access; hide system-level details from the application developer; and seamless scalability.__ Part of our work this semester will be to interact with these ideas in a practical way, not just a conceptual one.

### Q8 Tasks:
* __a) short response:__ What do Lin and Dyer consider the two features of an "ideal algorithm" from a scalability perspective?


* __b) short response:__ The mapper script below (created on the fly using a little Jupyter magic) will help us illustrate the concept of scalability. Run the provided code which passes this mapper script to our parallel computation 'framework' and runs the 'analysis' on the _Alice_ text file. Note that we've omitted a reducer for simplicity. What do you observe about the time it takes to run this "algorithm" when we use 1, 2 and 4 partitions? Does it meet Lin and Dyer's criteria?


* __c) short response:__ Let's try something similar with your Word Count analysis. Run the provided code to time your implementation with 1, 2, 4 and 8 partitions. What do you observe about the runtimes? Does this match your expectation? Speculate about why we might be seeing these times. What conclusions should we draw about the scalability of our implementation? [`HINT:` _consider the limitations of both your machine and our implementation... there are some competing forces at work, what are they?_]


* __d) OPTIONAL:__ Which components of your Map-Reduce algorithm are affected by a change in the number of partitions? Does increasing the number of partitions increase or decrease the total time spent on each of these portions of the task? What tradeoff does this cause?

### Q8 Student Answers:

> __a)__ First is Data: given twice the amount of data, the same algorithm should take at most twice as long to run. Second is Resources: given a cluster twice the size, the same algorithm should take no more than half as long to run. 

> __b)__ It almost is an "ideal algorithm" but not technically "ideal", when we split the file and pass them into the mappers, which is basically counting lines, the running time is reduced slightly above half. Theoretically, the mapper complexity is big $O(n)$, when split into $2^n$ files and run parallelly, the time should be reduced by $2^n$ times.

> __c)__ We see the time is almost the same with 1, 2, 8 partitions and time goes up with 16, 32 partitions. At the begining, I would think the time should goes down like the previous one, since the algorithm of reducer is also big $O(n)$ in terms of time complexity after every word is sorted. However, when we split the file, and each file count the words parallelly in mapper, the merged file will have more and more duplicated words as we have more temporary word count files. Therefore, it costs more running time to reduce. When we have 1, 2, 8 partitions, the time we saved on partition word counting is almost cancelled with the time with reducing the duplicated words; when we have 16 or 32 partitions, the time we saved is less then time in reducing.
The conculsion is there will be a partition number for our implementation, more time we saved on mapping, more time will be spent on reducing, similar to the bias variance trade off.

> __d)__ As explained on Q.8c, both mapping and reducing are affected by a change in the number of partitions. More time we saved on mapping, more time will be spent on reducing, similar to the bias variance trade off.

__Run the following cells to create the mapper referenced in `part b`__

In [64]:
!mkdir demo

In [65]:
%%writefile demo/mapper.py
#!/usr/bin/env python
"""
This mapper reads from STDIN and waits 0.001 seconds per line.
Its only purpose is to demonstrate one of the scalability ideas.
"""
import sys
import time
for line in sys.stdin:
    time.sleep(0.001)

Writing demo/mapper.py


In [66]:
# Make sure the mapper is executable
!chmod a+x demo/mapper.py

__Run the next three cells to apply our demo mapper with 1, 2 and 4 partitions.__

In [67]:
%%timeit
!./pWordCount_v2.sh 1 'data/alice.txt' 'demo/mapper.py'

4.47 s ± 7.11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [68]:
%%timeit
!./pWordCount_v2.sh 2 'data/alice.txt' 'demo/mapper.py'

2.31 s ± 2.33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [69]:
%%timeit
!./pWordCount_v2.sh 4 'data/alice.txt' 'demo/mapper.py'

1.23 s ± 4.74 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


__Run the following cells to repeat this process with your word count algorithm.__

In [87]:
%%timeit
!./pWordCount_v2.sh 1 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

196 ms ± 976 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [88]:
%%timeit
!./pWordCount_v2.sh 2 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

187 ms ± 443 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [89]:
%%timeit
!./pWordCount_v2.sh 4 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

194 ms ± 8.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [79]:
%%timeit
!./pWordCount_v2.sh 8 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

220 ms ± 2.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [84]:
%%timeit
!./pWordCount_v2.sh 16 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

285 ms ± 2.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [86]:
%%timeit
!./pWordCount_v2.sh 32 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

413 ms ± 3.39 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Question 9: Embarrassingly Parallel
`"If any one of them can explain it," said Alice, (she had grown so large in the last few minutes that she wasn’t a bit afraid of interrupting him,) "I’ll give him sixpence. I don’t believe there’s an atom of meaning in it."`
<div style="text-align: right">  -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 12  </div>

### Q9 Tasks:

* __a) short response:__ Describe what we mean by 'Embarrassingly Parallel' in terms of word counting. Does this term describe a 'task'? an 'implementation of a task'? 

* __b) OPTIONAL__: Define this concept in terms of 'associative' and 'commutative' operations. [`HINT:` _Refer to Week 2 Async, and the assigned_ [reading from EECS](https://patterns.eecs.berkeley.edu/?page_id=37) _about Map-Reduce Patterns_]

### Q9 Student Answers:

> __a)__ Embarrassingly Parallel can also be called 'Perfectly Parallel', and describes a task that can be perfectly decomposed into independent subtasks for which there is no dependency or need for communication between the subtasks or the results stemming from them. For word counting case, each counting mapper in each individual parition does not depend on any other partitions count aspect, and no communication is between paritions is required. The term "Embarrassingly Parallel" describes the implementation of a task in my opinion, as implementattion of parallel counting is the approach to tackle a task and a task itself can be considered broad.

> __b)__ Associative means that groupings do not matter: (a+b)+c = a+(b+c). Commutative means that order does not matter: a+b+c = a+c+b. Together, this implies that each component a, b, and c, as well as any groupings or orderings of those components, can be computed independently of each other. When aggregated or *combined* at the end, the result is still the same. A task executed in a manner that exhibits both commutative and associative operations would be said to be embarrassingly parallel.

### Congratulations, you have completed HW1! Please refer to the readme for submission instructions.

If you would like to provide feedback regarding this homework, please use the survey at: https://docs.google.com/forms/d/e/1FAIpQLSce9feiQeSkdP43A0ZYui1tMGIBfLfzb0rmgToQeZD9bXXX8Q/viewform